# Dallas 311 Request Frequency and Cedars Overlay




In [1]:
import pandas as pd
import geojson
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps
import requests

## Cedars mapping and basic info

Will contact Dallas Open Data for TIF coordinates, but 'eyeballing' for now from TIF map with google maps

In [2]:
cedarsBoundary = ([32.771868, -96.781941],
                  [32.773780, -96.783754],
                  [32.774646, -96.785771],
                  [32.773932, -96.789379],
                  [32.773954, -96.791344],
                  [32.773353, -96.795018],
                  [32.772538, -96.798182],
                  [32.772246, -96.798706],
                  [32.771174, -96.800133],
                  [32.765714, -96.792743],
                  [32.765254, -96.789466],
                  [32.768572, -96.785218],
                  [32.770256, -96.783397],
                  [32.771276, -96.782851],
                  [32.771868, -96.781941])

## Testing geoJSON from Socrata

Socrata offers geoJSON objects from their api, since we have configured our python environment with geopandas, this should in theory, make our life much easier

### Links
* 311 api endpoiint for 311 reports: https://www.dallasopendata.com/resource/hs2z-7kbd.geojson
* Example with geoJSON and folium, may be outdated: http://andrewgaidus.com/leaflet_webmaps_python/

### Data Exploration
First, I'll look at the geo data itself using the geoJSON endpoint for 311 calls

https://www.dallasopendata.com/resource/hs2z-7kbd.geojson

In [3]:
json311 = requests.get('https://www.dallasopendata.com/resource/hs2z-7kbd.geojson')

#Get lat long from original geometry

geo = (json311.json())

coords = []

for i in (j for j in geo['features'] if j is not None):
    try:
        coord = i['geometry']['coordinates']
        coords.append(i['geometry']['coordinates'])
    except:
        continue
        #will replace with NaN append later
        

coordsLatLong = []

for i in coords:
    coordsLatLong.append([i[1],i[0]])

In [5]:
cedarsMap = folium.Map(location=[32.770808, -96.790647],
              zoom_start = 15,
              tiles = "stamenterrain")


layerName = '311Events'
#Create empty lists to contain the point coordinates and the point pop-up information

pt_lyr = folium.FeatureGroup(name = layerName)
#FastMarkerCluster(data = coordsLatLong).add_to(pt_lyr)
MarkerCluster(coordsLatLong).add_to(pt_lyr)

    
pt_lyr.add_to(cedarsMap)


cedarsPolyGrp = folium.FeatureGroup(name = 'Cedars Region') 

folium.PolyLine(cedarsBoundary,
                fill = True,
                fill_color='#3186cc').add_to(cedarsPolyGrp)

cedarsPolyGrp.add_to(cedarsMap)


folium.LayerControl().add_to(cedarsMap)

cedarsMap